In [ ]:
#from IRBSubmission get all protocol IDs
#use multiRSCInformation api call with filter=parentStudyID eq ID to get submissions related to protocol
#use RadiationSafetySubmission api call with each RSC submission ID to get related information

#Table 1 <- information from each multiRSCInformation api call with parent protocol id as main key
#Table 2 <- information from each single radiationSafetySubmission api call with parent protocol id as main key

In [ ]:
import requests
import pandas as pd
import numpy as np
import json
import sys

requests.packages.urllib3.disable_warnings()

In [ ]:
api_multi = 'https://protect-training.cc.nih.gov/TRAINING-IRB/sd/PublicCustomLayouts/PSLib/WebApi/multiResult?interfaceID=MultiRSCInformation'

In [ ]:
session = requests.Session()
session.auth = (u<'ID'>, <'PWD'>)
session.verify = False

In [ ]:
response = session.get(api_multi)

In [ ]:
def json_flatten(json_dict):

    def inner_flatten(x, name=''):
        single_dict = {}
        if isinstance(x, dict):
            flattened_dict = {}
            for key,value in x.items():
                #recursively calling on deeper layers of the json
                #if 'safetySubmissionCustomExtension' not in key:
                #    flattened_x = inner_flatten(value, name + key + '_')
                #else:
                #    flattened_x = inner_flatten(value, name + key[32:] + '_')
                flattened_x = inner_flatten(value, key)
                flattened_dict.update(flattened_x)
            single_dict.update(flattened_dict)
                    
        elif isinstance(x, list):
            flattened_dict = {}
            flattened_x = []
            for obj in x:
                flattened_obj = inner_flatten(obj, name)
                flattened_x.append(flattened_obj)

            #will always be true due to the nature of the inner function return type
            if all(isinstance(y,dict) for y in flattened_x):
                for i, d in enumerate(flattened_x):
                    for k, v in d.items():
                        flattened_dict[f'{k}_{i+1}'] = v
                    
            single_dict.update(flattened_dict)
            
        else:
            #single_dict[name[:-1]] = x
            single_dict[name] = x
            
        return single_dict

    return inner_flatten(json_dict)

In [ ]:
IDs = set()
parentIDs = set()
multi_json = response.json()

In [ ]:
for json in multi_json:
    parentStudyID = json['parentStudyID']
    rscID = json['ID']

    parentIDs.add(parentStudyID)
    IDs.add(rscID)

In [ ]:
#loop through submissionIDs and perform single RS api call on them. insert json into singleRSC table
RSCSingleAPI = "https://protect-training.cc.nih.gov/TRAINING-IRB/sd/PublicCustomLayouts/PSLib/WebApi/singleResult?interfaceID=RadiationSafetySubmission&resourceID="

singleRSC_data = list()
for id in IDs:
    response = session.get(RSCSingleAPI + '\'' + id + '\'')
    singleRSC_data.append(json_flatten(response.json()['data']))
    
singleRSC_table = pd.DataFrame(singleRSC_data)

In [ ]:
#loop through parentIDs and perform multi RS api call on them. insert json into multiRSC table
RSCMultiAPI = "https://protect-training.cc.nih.gov/TRAINING-IRB/sd/PublicCustomLayouts/PSLib/WebApi/multiResult?interfaceID=MultiRSCInformation&$filter=parentStudyID eq "

multiRSC_data = list()
for id in parentIDs:
    response = session.get(RSCMultiAPI + '\'' + id + '\'')
    multiRSC_data = multiRSC_data + response.json()

multiRSC_table = pd.json_normalize(multiRSC_data)

In [1]:
import psycopg2
from sqlalchemy import create_engine

In [ ]:
db = create_engine('postgresql://<ID>:<PWD>@10.157.90.23:5432/PROTECT')
conn = db.connect()

In [ ]:
multiRSC_table.to_sql('RadiationSafetyStudyDetails', con=conn, if_exists='replace', index=FALSE)

In [ ]:
singleRSC_table.to_sql('RadiationSafetySubmissionDetails', con=conn, if_exists='replace', index=False)